# Coronavirus COVID19-Deaths

## Setup module, path and dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Import public module
import os
import sys
import time
import warnings

import pandas as pd

# setup paths
setPath = os.path.join(os.getenv(key='LinkPaths'), 'arrowSEnvs')
if setPath not in sys.path: sys.path.append(setPath)
from setupEnv import setupEnv
setupEnv.setPaths()
arrowDb = setupEnv.pathAt.get('path1')
COVID19DbasePath = setupEnv.pathAt.get('COVID19DbasePath')
localPathDataset = '__dataset___'

# Import Internal modul to manage the database, file database and path database
import arrowDb
from arrowDb.sets import SetPathFiles

In [3]:
# download datasets
urlFile = ['url/owid-covid-codebook.csv.url',
            'url/owid-covid-data.csv.url',
            'url/owid-covid-data.json.url',
            'url/owid-covid-data.xlsx.url']
SetPathFiles.url_download_file(urlFile, localPathDataset)

...Starting download for owid-covid-codebook.csv...
The file _dataset___\owid-covid-codebook.csv has been successfully downloaded.
The file _dataset___\owid-covid-data.csv already exists. Download skipped.
The file _dataset___\owid-covid-data.json already exists. Download skipped.
The file _dataset___\owid-covid-data.xlsx already exists. Download skipped.


In [4]:
# Setup the database connetion
SetPathFiles.list_files_path(localPathDataset)
COVID19db = arrowDb.JunctDbase(
                'sqlite',conn_typesub='sqlite3',
                file_dbase='owidcovid19.sqlite ',
                folder_path=COVID19DbasePath)
os.environ['COVID19dbstr'] = COVID19db.conn('conn string')

# load the magic sql
%load_ext sql
time.sleep(1)
%sql $COVID19dbstr

# Automatically limit the number of rows displayed
%config SqlMagic.displaylimit = 20

Files in path folder (extensions: all):


owid-covid-codebook.csv - 0.01MB (1.34e+01 KB)
owid-covid-data.csv - 98.12MB (1.00e+05 KB)
owid-covid-data.json - 227.30MB (2.33e+05 KB)
owid-covid-data.xlsx - 52.49MB (5.37e+04 KB)


Instance of the object has been created.
Connection (sqlite3) to 'owidcovid19.sqlite ' database, 'Open Connection' Successfully.


Connecting to 'sqlite:///C:\\\\GServer\\\\LinkPaths\\\\Datas\\\\DatashareWBox\\\\COVID19_dataset\\\\dbase\\owidcovid19.sqlite '

In [5]:
# print object list database
COVID19db.query('object list')

List for 'table' in 'owidcovid19.sqlite ' database:
- [Null] No table object in list


List for 'view' in 'owidcovid19.sqlite ' database:
- [Null] No view object in list


## Prepair Data

### Import 'owid-Covid-Data.csv' to Duckdb database

In [6]:
COVID19db.query(
    'import df to db',
    df=pd.read_csv(localPathDataset + '/owid-Covid-Data.csv'),
    table='owidCovidData',
    if_exists='replace')

The DataFrame has been successfully imported into table 'owidCovidData' in database(sqlite/sqlite3).


In [7]:
COVID19db.query("object list", type='table')

List for 'table' in 'owidcovid19.sqlite ' database:
- owidCovidData (table object)


#### Description the 'owidCovidData' table

In [8]:
COVID19db.query(
    'show descript table', 
    table='owidCovidData', 
    style='pr')

Description for 'owidCovidData'


+----+-----+--------------------------------------------+---------+---------+------------+----+
|    | cid |                    name                    |   type  | notnull | dflt_value | pk |
+----+-----+--------------------------------------------+---------+---------+------------+----+
| 1  |  0  |                  iso_code                  |   TEXT  |    0    |    None    | 0  |
| 2  |  1  |                 continent                  |   TEXT  |    0    |    None    | 0  |
| 3  |  2  |                  location                  |   TEXT  |    0    |    None    | 0  |
| 4  |  3  |                    date                    |   TEXT  |    0    |    None    | 0  |
| 5  |  4  |                total_cases                 |   REAL  |    0    |    None    | 0  |
| 6  |  5  |                 new_cases                  |   REAL  |    0    |    None    | 0  |
| 7  |  6  |             new_cases_smoothed             |   REAL  |    0    |    None    | 0  |
| 8  |  7  |                total_deaths

### Create view table ('covidDeaths') database from file .sql

In [9]:
COVID19db.query(
        'create view file',
        view='covidDeaths',
        file='covidDeaths.sql',
        path='sql',
        replace='yes')

View 'covidDeaths' has been created.
Total rows and columns: (429435, 59)


#### Description the 'covidDeaths' view table

In [10]:
COVID19db.query('show descript table', 
                table='covidDeaths', 
                style='pr')

Description for 'covidDeaths'


+----+-----+---------------------------------------+---------+---------+------------+----+
|    | cid |                  name                 |   type  | notnull | dflt_value | pk |
+----+-----+---------------------------------------+---------+---------+------------+----+
| 1  |  0  |                iso_code               |   TEXT  |    0    |    None    | 0  |
| 2  |  1  |               continent               |   TEXT  |    0    |    None    | 0  |
| 3  |  2  |                location               |   TEXT  |    0    |    None    | 0  |
| 4  |  3  |                  date                 |   TEXT  |    0    |    None    | 0  |
| 5  |  4  |              total_cases              |   REAL  |    0    |    None    | 0  |
| 6  |  5  |               new_cases               |   REAL  |    0    |    None    | 0  |
| 7  |  6  |           new_cases_smoothed          |   REAL  |    0    |    None    | 0  |
| 8  |  7  |              total_deaths             |   REAL  |    0    |    None    | 0  |

### Create view table ('covidVaccinations') database from file .sql


In [11]:
COVID19db.query('create view file',
                view='covidVaccinations', 
                file='covidVaccinations.sql',
                path='sql',
                replace=True)

View 'covidVaccinations' has been created.
Total rows and columns: (429435, 37)


#### Description the 'covidVaccinations' view table


In [12]:
COVID19db.query('show descript table', 
                table='covidVaccinations', 
                style='pr')

Description for 'covidVaccinations'


+----+-----+---------------------------------------+------+---------+------------+----+
|    | cid |                  name                 | type | notnull | dflt_value | pk |
+----+-----+---------------------------------------+------+---------+------------+----+
| 1  |  0  |                iso_code               | TEXT |    0    |    None    | 0  |
| 2  |  1  |               continent               | TEXT |    0    |    None    | 0  |
| 3  |  2  |                location               | TEXT |    0    |    None    | 0  |
| 4  |  3  |                  date                 | TEXT |    0    |    None    | 0  |
| 5  |  4  |               new_tests               | REAL |    0    |    None    | 0  |
| 6  |  5  |              total_tests              | REAL |    0    |    None    | 0  |
| 7  |  6  |        total_tests_per_thousand       | REAL |    0    |    None    | 0  |
| 8  |  7  |         new_tests_per_thousand        | REAL |    0    |    None    | 0  |
| 9  |  8  |           new_tests

### Query to database

In [13]:
%%sql $COVID19dbstr

SELECT *
FROM covidDeaths
ORDER BY 4

LIMIT 5

iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
ARG,South America,Argentina,2020-01-01,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,4.0,4.0,0.0,0.0,None,None,None,None,tests performed,None,None,None,None,None,None,None,None,None,0.0,45510324,16.177,31.9,11.198,7.441,18933.907,0.6,191.032,5.5,16.2,27.7,None,5.0,76.67,0.845
MEX,North America,Mexico,2020-01-01,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,25.0,25.0,0.0,0.0,None,None,None,None,people tested,None,None,None,None,None,None,None,None,None,0.0,127504120,66.444,29.3,6.857,4.321,17336.469,2.5,152.783,13.06,6.9,21.4,87.847,1.38,75.05,0.779
ARG,South America,Argentina,2020-01-02,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,95.0,99.0,0.002,0.002,None,None,None,None,tests performed,None,None,None,None,None,None,None,None,None,0.0,45510324,16.177,31.9,11.198,7.441,18933.907,0.6,191.032,5.5,16.2,27.7,None,5.0,76.67,0.845
MEX,North America,Mexico,2020-01-02,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,72.0,97.0,0.001,0.001,None,None,None,None,people tested,None,None,None,None,None,None,None,None,None,0.0,127504120,66.444,29.3,6.857,4.321,17336.469,2.5,152.783,13.06,6.9,21.4,87.847,1.38,75.05,0.779
ARG,South America,Argentina,2020-01-03,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,5.0,104.0,0.002,0.0,None,None,None,None,tests performed,None,None,None,None,None,None,None,None,None,0.0,45510324,16.177,31.9,11.198,7.441,18933.907,0.6,191.032,5.5,16.2,27.7,None,5.0,76.67,0.845


In [14]:
%%sql $COVID19dbstr

SELECT *
FROM covidVaccinations
ORDER BY 3

LIMIT 5

iso_code,continent,location,date,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
AFG,Asia,Afghanistan,2020-01-05,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.0,54.422,18.6,2.581,1.337,1803.987,None,597.029,9.59,None,None,37.746,0.5,64.83,0.511
AFG,Asia,Afghanistan,2020-01-06,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.0,54.422,18.6,2.581,1.337,1803.987,None,597.029,9.59,None,None,37.746,0.5,64.83,0.511
AFG,Asia,Afghanistan,2020-01-07,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.0,54.422,18.6,2.581,1.337,1803.987,None,597.029,9.59,None,None,37.746,0.5,64.83,0.511
AFG,Asia,Afghanistan,2020-01-08,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.0,54.422,18.6,2.581,1.337,1803.987,None,597.029,9.59,None,None,37.746,0.5,64.83,0.511
AFG,Asia,Afghanistan,2020-01-09,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.0,54.422,18.6,2.581,1.337,1803.987,None,597.029,9.59,None,None,37.746,0.5,64.83,0.511


## Built Dataframe from file .sql

In [15]:
COVID19db.query("query file",
                file='covidDeathsTcasesTdeaths.sql',
                path='sql',
                nrows=10)

,location,Year,Month,YearMonth,TotalCases,TotalDeaths,DeathPercentage,CasePopulationPercentage,DeathPopulationPercentage,population
1,Afghanistan,2020,01,2020-01,0.0,0.0,None,0.0,0.0,41128772
2,Africa,2020,01,2020-01,0.0,0.0,None,0.0,0.0,1426736614
3,Albania,2020,01,2020-01,0.0,0.0,None,0.0,0.0,2842318
4,Algeria,2020,01,2020-01,0.0,0.0,None,0.0,0.0,44903228
5,American Samoa,2020,01,2020-01,0.0,0.0,None,0.0,0.0,44295
6,Andorra,2020,01,2020-01,0.0,0.0,None,0.0,0.0,79843
7,Angola,2020,01,2020-01,0.0,0.0,None,0.0,0.0,35588996
8,Anguilla,2020,01,2020-01,0.0,0.0,None,0.0,0.0,15877
9,Antigua and Barbuda,2020,01,2020-01,0.0,0.0,None,0.0,0.0,93772
10,Argentina,2020,01,2020-01,0.0,0.0,None,0.0,0.0,45510324


The table result display in 'DataFrame'
Limits the total rows(nrows=) to 10 rows
Total rows and columns: (14105, 10)


In [16]:
df = COVID19db.query('get df file', file='covidDeathsTcasesTdeaths.sql', path='sql')
df.fillna(0, inplace=True)
df.sort_values('YearMonth', inplace=True)
display(df.info())
display(df.head())

DataFrame has been successfully created from the SQL string.
	Total rows and columns: (14105, 10)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 14105 entries, 1 to 10065
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   location                   14105 non-null  object 
 1   Year                       14105 non-null  object 
 2   Month                      14105 non-null  object 
 3   YearMonth                  14105 non-null  object 
 4   TotalCases                 14105 non-null  float64
 5   TotalDeaths                14105 non-null  float64
 6   DeathPercentage            14105 non-null  float64
 7   CasePopulationPercentage   14105 non-null  float64
 8   DeathPopulationPercentage  14105 non-null  float64
 9   population                 14105 non-null  int64  
dtypes: float64(5), int64(1), object(4)
memory usage: 1.2+ MB


None

,location,Year,Month,YearMonth,TotalCases,TotalDeaths,DeathPercentage,CasePopulationPercentage,DeathPopulationPercentage,population
1,Afghanistan,2020,01,2020-01,0.0,0.0,0.0,0.0,0.0,41128772
158,Nicaragua,2020,01,2020-01,0.0,0.0,0.0,0.0,0.0,6948395
159,Niger,2020,01,2020-01,0.0,0.0,0.0,0.0,0.0,26207982
160,Nigeria,2020,01,2020-01,0.0,0.0,0.0,0.0,0.0,218541216
161,Niue,2020,01,2020-01,0.0,0.0,0.0,0.0,0.0,1952


In [17]:
df = COVID19db.query('get df file', file='covidDeathsTcasesTdeaths.sql', path='sql', confirm=False)
df.fillna(0, inplace=True)
df.sort_values('YearMonth', inplace=True)

## Plot Data

In [18]:
from scripts.Plots import Plots as plots

### Total Cases Vs Total Deaths 

In [22]:
xcol='YearMonth'
xlabel='Year-Month'
ycol1 = 'TotalCases'
ylabel1 = 'Total Cases'
ycol2 = 'TotalDeaths'
ylabel2 = 'Total Deaths'
line_col = "location"
line_selc = ["Indonesia", "Singapore", "Malaysia","Thailand","Philippines","Vietnam","Laos",]
style="plotly"

plots.lines_inplot(df=df,
                xcol=xcol,
                xlabel=xlabel,
                ycol1=ycol1,
                ylabel1=ylabel1,
                ycol2=ycol2,
                ylabel2=ylabel2,
                line_col=line_col,
                line_selc=line_selc,
                style=style,
                split=True)

### Total Cases

In [23]:
tlabel="Total Cases Per Year and Month for selected locations"
xcol='YearMonth'
xlabel='Year-Month'
ycol = 'TotalCases'
ylabel ='Total Cases'
line_col = "location"
line_selc = ["Indonesia", "Singapore", "Malaysia","Thailand","Philippines","Vietnam","Laos",]
style="plotly"

plots.periodic_month(df=df,
                xcol=xcol,
                xlabel=xlabel,
                ycol=ycol,
                ylabel=ylabel,
                tlabel=tlabel,
                style=style,
                line_col=line_col,
                line_selc=line_selc)

### Total Deaths

In [21]:
tlabel="Total deaths per Year and Month for selected locations"
xcol='YearMonth'
xlabel='Year-Month'
ycol = 'TotalDeaths'
ylabel ='Total Deaths'
line_col = "location"
line_selc = ["Indonesia", "Singapore", "Malaysia","Thailand","Philippines","Vietnam","Laos",]
style="plotly"

plots.periodic_month(df=df,
                xcol=xcol,
                xlabel=xlabel,
                ycol=ycol,
                ylabel=ylabel,
                tlabel=tlabel,
                style=style,
                line_col=line_col,
                line_selc=line_selc)